In [ ]:
import requests
import json
import pandas as pd
import io

In [ ]:
request_url = 'https://www.regionalstatistik.de/genesis/online'
request_params = {
    "operation": "download",
    "code": "12411-01-01-4",
    "startjahr":"",
    "endjahr": "",
    "zeiten": "",
    "option":"csv", #(csv, ffcsv, xlsx, xml)
    "zeitscheiben": "2",  #Anzahl der aktuellsten Zeitscheiben, die tabelliert werden sollen
    "language": "de"
}
# save the response
de_response = requests.get(url=request_url, params=request_params)
de_response

In [ ]:
de_response.content